In [1]:
import sys

In [2]:
print(sys.path)

['/Users/khant/Documents/UFZ/Git/iasdt-workflows/notebooks', '/Users/khant/opt/anaconda3/lib/python39.zip', '/Users/khant/opt/anaconda3/lib/python3.9', '/Users/khant/opt/anaconda3/lib/python3.9/lib-dynload', '', '/Users/khant/.local/lib/python3.9/site-packages', '/Users/khant/opt/anaconda3/lib/python3.9/site-packages', '/Users/khant/opt/anaconda3/lib/python3.9/site-packages/aeosa']


In [3]:
# some_file.py
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
if "../src/data/" in sys.path:
    print("Path not needed")
else:
    sys.path.insert(1, "../src/data/")


In [7]:
from chelsa import download_data
#down = wget.download("https://os.zhdk.cloud.switch.ch/envicloud/chelsa/chelsa_V2/GLOBAL/annual/swb/CHELSA_swb_2013_V.2.1.tif")
download_data("../references/chelsa_paths.txt", "../data/raw/chelsa/")

In [1]:
import xarray as xr

data_array = xr.open_rasterio("../data/raw/chelsa/CHELSA_swb_2013_V.2.1.tif", engine="scipy")
print(data_array)

/var/folders/pd/scby1cjx6tx_33rnmv0q6r8m0000gr/T/ipykernel_57793/876188145.py:3: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  data_array = xr.open_rasterio("../data/raw/chelsa/CHELSA_swb_2013_V.2.1.tif", engine="scipy")


<xarray.DataArray (band: 1, y: 20880, x: 43200)>
[902016000 values with dtype=int16]
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 84.0 83.99 83.98 83.97 ... -89.97 -89.98 -89.99 -90.0
  * x        (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0 180.0
Attributes:
    transform:      (0.0083333333, 0.0, -180.00013888885002, 0.0, -0.00833333...
    crs:            +init=epsg:4326
    res:            (0.0083333333, 0.0083333333)
    is_tiled:       0
    nodatavals:     (-32767.0,)
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area


In [51]:
ds = data_array.to_dataset('band')
ds = ds.rename({1: '1'})

In [55]:
ds.to_netcdf("../data/interim/CHELSA_swb_2013.nc", group="CHELSA_swb_2013/", mode="w")

NameError: name 'netCDF4' is not defined